# Loading Responses

In [1]:
options(warn=1)
if(!require('lme4')) {
    install.packages('lme4', repos='https://cloud.r-project.org')
    library('lme4')
}
if(!require('readr')) {
    install.packages('readr', repos='https://cloud.r-project.org')
    library('readr')
}
if(!require('marginaleffects')) {
    install.packages('marginaleffects', repos='https://cloud.r-project.org')
    library('marginaleffects')
}
if(!require('emmeans')) {
    install.packages('emmeans', repos='https://cloud.r-project.org')
    library('emmeans')
}
if(!require('car')) {
    install.packages('car', repos='https://cloud.r-project.org')
    library('car')
}
df <- read_csv("gaze-analysis/result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df[df$Group == "glm-4v-9b" & df$list_id != -1 & df$Part != "p0", ]
df$offset <- log(1/df$n_candidates / (1 - 1/df$n_candidates))
df$Proximity <- df$Proximity - mean(df$Proximity)
df$n_candidates <- df$n_candidates - mean(df$n_candidates)

Loading required package: lme4

Loading required package: Matrix

Loading required package: readr

Loading required package: marginaleffects

Loading required package: emmeans

Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'

Loading required package: car

Loading required package: carData

Rows: 156780 Columns: 14
── Column specification ────────────────────────────────────────────────────────────────
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# Fitting models

In [2]:
model1 <- glmer(Accuracy ~ Angle + Proximity + n_candidates + Actor + (1|Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
“Model failed to converge with max|grad| = 0.0355467 (tol = 0.002, component 1)”


In [3]:
model2 <- glmer(Accuracy ~ Angle + Proximity + n_candidates + Actor + (1|Actor:Candidates) + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

boundary (singular) fit: see help('isSingular')



In [4]:
model3 <- glmer(Accuracy ~ Angle + Proximity + n_candidates + Actor + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)

# Model selection

In [5]:
model <- model3

# Interpretation

In [6]:
summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + Proximity + n_candidates + Actor + (1 | Stimulus_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  6195.0   6244.8  -3090.5   6181.0     8993 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-2.99492 -0.13328 -0.06035  0.15027  3.09692 

Random effects:
 Groups      Name        Variance Std.Dev.
 Stimulus_ID (Intercept) 25.23    5.023   
Number of obs: 9000, groups:  Stimulus_ID, 900

Fixed effects:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)    0.5260     0.3748   1.404  0.16043    
Angleleft     -0.3161     0.4879  -0.648  0.51703    
Angleright    -0.2926     0.4889  -0.598  0.54954    
Proximity     -0.7257     0.2490  -2.915  0.00356 ** 
n_candidates  -1.4594     0.2936  -4.970 6.69e-07 ***
ActorY        -1.3476     0.4177  -3.226  0.00125 ** 
---
Signif. codes:

In [7]:
log_odds_means <- emmeans(model, spec = "Angle")
print(log_odds_means)
pairs(log_odds_means)
log_odds_means <- emmeans(model, spec = "Angle", regrid = "response")
print(log_odds_means)
pairs(log_odds_means)
               
vc_list <- lapply(VarCorr(model), function(x) attr(x, "stddev")^2)
vc <- sum(unlist(vc_list))
effect_size_denom_t <- sqrt(vc + pi^2/3)

res <- summary(model)$coefficients
res <- as.data.frame(res)
res <- cbind(term = rownames(res), res)
rownames(res) <- NULL
res$effect_size_denom <- effect_size_denom_t
res$effect_size <- res$Estimate / effect_size_denom_t
res

 Angle emmean    SE  df asymp.LCL asymp.UCL
 front -0.498 0.336 Inf     -1.16    0.1605
 left  -0.814 0.363 Inf     -1.53   -0.1034
 right -0.791 0.365 Inf     -1.51   -0.0755

Results are averaged over the levels of: Actor 
Results are given on the logit (not the response) scale. 
Confidence level used: 0.95 


 contrast      estimate    SE  df z.ratio p.value
 front - left    0.3161 0.488 Inf   0.648  0.7935
 front - right   0.2926 0.489 Inf   0.598  0.8209
 left - right   -0.0236 0.506 Inf  -0.047  0.9988

Results are averaged over the levels of: Actor 
Results are given on the log odds ratio (not the response) scale. 
P value adjustment: tukey method for comparing a family of 3 estimates 

 Angle  prob     SE  df asymp.LCL asymp.UCL
 front 0.390 0.0716 Inf     0.250     0.530
 left  0.325 0.0721 Inf     0.183     0.466
 right 0.329 0.0726 Inf     0.187     0.472

Results are averaged over the levels of: Actor 
Confidence level used: 0.95 


 contrast      estimate    SE  df z.ratio p.value
 front - left   0.06558 0.101 Inf   0.650  0.7922
 front - right  0.06086 0.101 Inf   0.601  0.8195
 left - right  -0.00472 0.101 Inf  -0.047  0.9988

Results are averaged over the levels of: Actor 
P value adjustment: tukey method for comparing a family of 3 estimates 

term,Estimate,Std. Error,z value,Pr(>|z|),effect_size_denom,effect_size
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.5260132,0.3747527,1.4036275,1.604298e-01,5.340172,0.09850117
Angleleft,-0.3161462,0.4879364,-0.6479251,5.170334e-01,5.340172,-0.05920150
Angleright,-0.2925773,0.4888882,-0.5984545,5.495367e-01,5.340172,-0.05478800
Proximity,-0.7257426,0.2489979,-2.9146534,3.560838e-03,5.340172,-0.13590248
n_candidates,-1.4594303,0.2936401,-4.9701329,6.690701e-07,5.340172,-0.27329276
ActorY,-1.3476203,0.4176765,-3.2264689,1.253278e-03,5.340172,-0.25235523
